## Description 

In this notebook, we will pull Alabama's raw assessment files which we have downloaded and converted to CSV manually. We convert the CSV manually because these have merged cells.

In [1]:
import glob 
import csv
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)

## Pull Data From Folder
https://www.alabamaachieves.org/reports-data/school-performance/#2021_-_2022

https://eddir.alsde.edu/SiteInfo/

https://reportcard.alsde.edu/SupportingData_Accountability.aspx

In [2]:
all_raw_xls_files = glob.glob('../raw_data_files_clean/*')
all_raw_xls_files.sort()

all_raw_xls_files

['../raw_data_files_clean/adm_2020.csv',
 '../raw_data_files_clean/adm_2021.csv',
 '../raw_data_files_clean/adm_2022.csv',
 '../raw_data_files_clean/assessment_ela_2015.csv',
 '../raw_data_files_clean/assessment_ela_2016.csv',
 '../raw_data_files_clean/assessment_ela_2017.csv',
 '../raw_data_files_clean/assessment_ela_2018.csv',
 '../raw_data_files_clean/assessment_ela_2019.csv',
 '../raw_data_files_clean/assessment_ela_2021.csv',
 '../raw_data_files_clean/assessment_ela_2022.csv',
 '../raw_data_files_clean/assessment_math_2015.csv',
 '../raw_data_files_clean/assessment_math_2016.csv',
 '../raw_data_files_clean/assessment_math_2017.csv',
 '../raw_data_files_clean/assessment_math_2018.csv',
 '../raw_data_files_clean/assessment_math_2019.csv',
 '../raw_data_files_clean/assessment_math_2021.csv',
 '../raw_data_files_clean/assessment_math_2022.csv',
 '../raw_data_files_clean/assessmentparticipation_2021.csv',
 '../raw_data_files_clean/assessmentparticipation_2022.csv',
 '../raw_data_files_

In [3]:
##  Pull files and create dataframes 
def pull_files_and_create_df(raw_file_names): 
    df_list=[]
    for i in raw_file_names:
        name = i.split('/')[2].split('.')[0]
        globals()[f'df_{name}'] = pd.read_csv(i, low_memory=False)
        d = df_list.append(f'df_{name}')
    df_list.sort(reverse=True)
    return df_list

In [4]:
## Match file column names 
def match_files(list_of_file_names):  
    
    # change column names to lowercase and replace space with underscores
    for d in list_of_file_names: 
        globals()[d].columns = list((map(lambda x: x.lower().replace(' ', '_'),list(globals()[d]))))
    
    # create matching dataframe 
    df=[]
    for n in range(len(list_of_file_names)):
        name = list_of_file_names[n]
        total_num_cols1 = len(globals()[list_of_file_names[n]].columns)
        set_f1_cols = set((map(lambda x: "'" + x + "'", list(globals()[list_of_file_names[n]]))))
               
        if n+1 == len(list_of_file_names):
            name0 = 'NA'
            total_num_cols0 = 0
            set_f0_cols = []        
        else:
            name0 = list_of_file_names[n+1]
            total_num_cols0 = len(globals()[list_of_file_names[n+1]].columns)
            set_f0_cols = set((map(lambda x: "'" + x + "'", list(globals()[list_of_file_names[n+1]]))))
        dict = {'current_file_name': name,
                'prev_file_name': name0,
                'current_num_cols': total_num_cols1,
                'prev_num_cols': total_num_cols0,
                'current_cols': sorted(set_f1_cols),
                'prev_cols': sorted(set_f0_cols),
                'matching_columns': sorted(set_f1_cols.intersection(set_f0_cols)),
                'num_matching': len(set_f1_cols.intersection(set_f0_cols)),
                'mismatched_columns': sorted(set_f1_cols.difference(set_f0_cols)),
                'num_mismatching': len(set_f1_cols.difference(set_f0_cols))
                }
        df.append(dict)
    df_chx = pd.DataFrame(df)
    return df_chx

In [60]:
def df_to_change_and_merge(df_name_list, column_list): 
    appended_data = []
    
    df_name = df_name_list[0].split('_')[0]+'_'+df_name_list[0].split('_')[1]
    df_files = [names for names in df_name_list if names.startswith(df_name)]
    # df_files = df_name_list
 
    for d in df_files:
        # print(globals()[d])
        globals()['updated_'+d] = globals()[d].copy()
        # try:
        globals()['updated_'+d] = globals()['updated_'+d].rename(columns={'subpopulation':'sub_population', 'percent_proficient':'proficient_rate'})
        globals()['updated_'+d] = globals()['updated_'+d].loc[(globals()[d]['gender']=='All Gender')
        #                                                  & (globals()['updated_'+d]['race']=='All Race')
        #                                                  & (globals()['updated_'+d]['ethnicity']=='All Ethnicity')
        #                                                  & (globals()['updated_'+d]['sub_population']=='All SubPopulation')
                                       ]    
        # except:
        #     pass
        globals()['updated_'+d] = globals()['updated_'+d][column_list]
        globals()['updated_'+d]['file_source'] = d
        globals()['updated_'+d]['file_year'] = d.split('_')[-1]
        globals()['updated_'+d] = globals()['updated_'+d] 
        appended_data.append(globals()['updated_'+d])
    
    globals()[df_name] = pd.concat(appended_data)
    try:
        globals()[df_name] = pd.merge(globals()[df_name], df_directory, on=['system_name', 'school_name', 'file_year'], how='left')
    except:
        globals()[df_name] = pd.merge(globals()[df_name], df_directory, on=['system_code', 'school_code', 'file_year'], how='left')
    globals()[df_name].to_csv('../raw_data_files_clean/merged_{}.csv'.format(df_name[3:]), encoding='utf-8', index=False)

In [6]:
cleaned_csv_files = glob.glob('../raw_data_files_clean/*.csv')
cleaned_csv_files.sort()
cleaned_csv_files[:5]

['../raw_data_files_clean/adm_2020.csv',
 '../raw_data_files_clean/adm_2021.csv',
 '../raw_data_files_clean/adm_2022.csv',
 '../raw_data_files_clean/assessment_ela_2015.csv',
 '../raw_data_files_clean/assessment_ela_2016.csv']

In [7]:
%%time
df_all_files = pull_files_and_create_df(cleaned_csv_files)

CPU times: user 52.3 s, sys: 12.6 s, total: 1min 4s
Wall time: 1min 13s


In [8]:
df_file_name_specs_check = match_files(df_all_files)
pd.set_option('display.max_colwidth', None)
# df_file_name_specs_check

In [9]:
df_all_files

['df_merged_grad',
 'df_merged_directory',
 'df_merged_demographics',
 'df_merged_assessment_all_students',
 'df_merged_assessment_2022',
 'df_merged_assessment',
 'df_merged_adm',
 'df_grad_2022',
 'df_grad_2021',
 'df_grad_2020',
 'df_grad_2019',
 'df_grad_2018',
 'df_from_file_assessment_math_2022',
 'df_from_file_assessment_ela_2022',
 'df_frl_2022',
 'df_directory_2022',
 'df_directory_2021',
 'df_directory_2020',
 'df_directory_2019',
 'df_directory_2018',
 'df_directory_2017',
 'df_directory_2016',
 'df_directory_2015',
 'df_demographics_2022',
 'df_demographics_2021',
 'df_demographics_2020',
 'df_demographics_2019',
 'df_demographics_2018',
 'df_demographics_2017',
 'df_demographics_2016',
 'df_demographics_2015',
 'df_attendance_2022',
 'df_assessmentparticipation_2022',
 'df_assessmentparticipation_2021',
 'df_assessment_math_2022',
 'df_assessment_math_2021',
 'df_assessment_math_2019',
 'df_assessment_math_2018',
 'df_assessment_math_2017',
 'df_assessment_math_2016',
 'df

In [10]:
df_all_files.remove('df_merged_assessment_2022')
df_all_files.remove('df_adm_2022')
df_all_files.remove('df_adm_2021')
df_all_files.remove('df_adm_2020')
# df_all_files.remove('df_grad_2022')
# df_all_files.remove('df_grad_2021')
# df_all_files.remove('df_grad_2020')
df_all_files

['df_merged_grad',
 'df_merged_directory',
 'df_merged_demographics',
 'df_merged_assessment_all_students',
 'df_merged_assessment',
 'df_merged_adm',
 'df_grad_2022',
 'df_grad_2021',
 'df_grad_2020',
 'df_grad_2019',
 'df_grad_2018',
 'df_from_file_assessment_math_2022',
 'df_from_file_assessment_ela_2022',
 'df_frl_2022',
 'df_directory_2022',
 'df_directory_2021',
 'df_directory_2020',
 'df_directory_2019',
 'df_directory_2018',
 'df_directory_2017',
 'df_directory_2016',
 'df_directory_2015',
 'df_demographics_2022',
 'df_demographics_2021',
 'df_demographics_2020',
 'df_demographics_2019',
 'df_demographics_2018',
 'df_demographics_2017',
 'df_demographics_2016',
 'df_demographics_2015',
 'df_attendance_2022',
 'df_assessmentparticipation_2022',
 'df_assessmentparticipation_2021',
 'df_assessment_math_2022',
 'df_assessment_math_2021',
 'df_assessment_math_2019',
 'df_assessment_math_2018',
 'df_assessment_math_2017',
 'df_assessment_math_2016',
 'df_assessment_math_2015',
 'df_a

In [11]:
df_directory_2022['school_name'] = (
    df_directory_2022['school_name']
    .replace('i3 Academy','i3 Academy - LEA')
    .replace('LEAD Academy','LEAD Academy - LEA')
)

df_demographics_2022['school'] = (
    df_demographics_2022['school']
    .replace('i3 Academy','i3 Academy - LEA')
    .replace('LEAD Academy','LEAD Academy - LEA')
)
                    
for x in df_all_files:
    globals()[x] = globals()[x].rename(columns={'system':'system_name','school':'school_name'})

    globals()[x]['school_name'] = (globals()[x]['school_name']
                    .replace('i3 Academy','i3 Academy - Elementary School')
                    .replace('i3 Academy Phase 1','i3 Academy - Elementary School')
                    .replace('i3 Academy Phase 2','i3 Academy - Middle School')
                    .replace('LEAD Academy','LEAD Academy - Elementary School')
                    .replace('LEAD Academy Building A','LEAD Academy - Elementary School')
                    .replace('LEAD Academy Building B','LEAD Academy - Middle School'))

    globals()[x]['system_name'] = (globals()[x]['system_name'].replace('MAEF Public Charter Schools','Acceleration Day and Evening Academy'))
    print(x)    

    
    ## 'i3 Academy' and 'i3 Academy Phase 1' should be 'i3 Academy - Elementary School' 
    ## 'i3 Academy Phase 2' should be 'i3 Academy - Middle School' 
    ## 'LEAD Academy' and 'LEAD Academy Building A' should be 'LEAD Academy - Elementary School' 
    ## 'LEAD Academy Building B' should be 'LEAD Academy - Middle School'
    ## 'MAEF Public Charter Schools'	'Acceleration Day and Evening Academy'


df_merged_grad
df_merged_directory
df_merged_demographics
df_merged_assessment_all_students
df_merged_assessment
df_merged_adm
df_grad_2022
df_grad_2021
df_grad_2020
df_grad_2019
df_grad_2018
df_from_file_assessment_math_2022
df_from_file_assessment_ela_2022
df_frl_2022
df_directory_2022
df_directory_2021
df_directory_2020
df_directory_2019
df_directory_2018
df_directory_2017
df_directory_2016
df_directory_2015
df_demographics_2022
df_demographics_2021
df_demographics_2020
df_demographics_2019
df_demographics_2018
df_demographics_2017
df_demographics_2016
df_demographics_2015
df_attendance_2022
df_assessmentparticipation_2022
df_assessmentparticipation_2021
df_assessment_math_2022
df_assessment_math_2021
df_assessment_math_2019
df_assessment_math_2018
df_assessment_math_2017
df_assessment_math_2016
df_assessment_math_2015
df_assessment_ela_2022
df_assessment_ela_2021
df_assessment_ela_2019
df_assessment_ela_2018
df_assessment_ela_2017
df_assessment_ela_2016
df_assessment_ela_2015


In [12]:
# df_assessment_ela_2022.loc[
#     # (df_assessment_ela_2022['subpopulation']=='Students with Limited English Proficiency') 
# # & 
#     (df_assessment_ela_2022['system_name']=='i3 Academy')
# & (df_assessment_ela_2022['grade']=='ALL')
# & (df_assessment_ela_2022['gender']=='All Gender')
# & (df_assessment_ela_2022['race']=='All Race')
# & (df_assessment_ela_2022['subpopulation']=='All SubPopulation')
# & (df_assessment_ela_2022['ethnicity']=='All Ethnicity')]

In [13]:
df_district_charter_mapping = pd.DataFrame({'system_code':
    [800,51,802,802,808,801,803,805,805,810],
    'school_code':
    [10,220,10,15,10,10,10,10,15,10],
    'updated_district':
    ['Mobile County','Montgomery County','Montgomery County','Montgomery County','Perry County','Sumter County','Birmingham City','Birmingham City','Birmingham City','Homewood City'],
    'updated_district_code':[49,51,51,51,53,60,114,114,114,157]
})

In [14]:
for i in [#df_assessment_math_2022,df_assessment_ela_2022,df_assessmentparticipation_2022,
    df_directory_2022,
          df_directory_2021,
          df_directory_2020,
          df_directory_2019,
          df_directory_2018,
          df_directory_2017,
          df_directory_2016,
          df_directory_2015]:
    i['school_name'] = np.where(i['school_code'].astype('int64')==1,i['system_name'],
                                np.where(i['school_code'].astype('int64')==0,i['system_name'],i['school_name']))

In [15]:
df_assessment_math_2022.loc[df_assessment_math_2022['system_name']=='Sumter County']

,year,system_name,school_name,subject,grade,gender,race,ethnicity,sub_population,enrolled,tested,proficient,participation_rate,proficient_rate,level_1,level_2,level_3,level_4,level_1_%,level_2_%,level_3_%,level_4_%
203435,2022,Sumter County,Sumter County,Math,All Grades,All Gender,All Race,All Ethnicity,All SubPopulation,496.35,482.00,*,~,*,347,127,*,*,71.99,26.35,*,0.00
203436,2022,Sumter County,Sumter County,Math,All Grades,All Gender,All Race,All Ethnicity,Students with Disabilities,61.00,*,*,*,*,51,*,*,*,80.95,*,*,0.00
203437,2022,Sumter County,Sumter County,Math,All Grades,All Gender,All Race,All Ethnicity,General Education Students,435.35,419.00,*,~,*,296,118,*,*,70.64,28.16,*,0.00
203438,2022,Sumter County,Sumter County,Math,All Grades,All Gender,All Race,All Ethnicity,Economically Disadvantaged,369.67,357.00,*,~,*,259,90,*,*,72.55,25.21,*,0.00
203439,2022,Sumter County,Sumter County,Math,All Grades,All Gender,All Race,All Ethnicity,Military Family,*,*,*,*,0.00,*,*,*,*,*,*,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
636157,2022,Sumter County,Sumter Central High School,Math,Grade 11,Male,Black or African American,Other Ethnicity,All SubPopulation,29.00,*,*,*,0.00,*,*,*,*,*,*,0.00,0.00
636158,2022,Sumter County,Sumter Central High School,Math,Grade 11,Male,Black or African American,Other Ethnicity,Students with Disabilities,*,*,*,*,0.00,*,*,*,*,*,0.00,0.00,0.00
636159,2022,Sumter County,Sumter Central High School,Math,Grade 11,Male,Black or African American,Other Ethnicity,General Education Students,24.00,*,*,*,0.00,*,*,*,*,*,*,0.00,0.00
636160,2022,Sumter County,Sumter Central High School,Math,Grade 11,Male,Black or African American,Other Ethnicity,Economically Disadvantaged,24.00,*,*,*,0.00,*,*,*,*,*,*,0.00,0.00


In [16]:
directory_list = ['df_directory_2022',
                  'df_directory_2021',
                  'df_directory_2020',
                  'df_directory_2019',
                  'df_directory_2018',
                  'df_directory_2017',
                  'df_directory_2016',
                  'df_directory_2015']
for d in directory_list:
    globals()[d]['is_charter'] = 'False'
    globals()[d]['is_charter'] = (
        globals()[d]['site_type']
        .replace('Public Charter School - Converted','True')
        .replace('Public Charter School Startup','True')
        .replace('Public Regular School','False')
        .replace('Public Virtual School','False')
        .replace('Public Magnet School','False')
        .replace('Public Alternative School','False')
        .replace('Public Special Education School','False')
        .replace('Public Board of Education',np.NaN)
        .replace('Central Office',np.NaN)
        .replace('State Board District',np.NaN)
        .replace('Regular School with Magnet Program','False')
        .replace('Special Education School (No Detail)','False')
        .replace('Special Education - Private Services','False')
        .replace('Public Career & Technical Center','False')
        .replace('State Supported School Other Facility','False')
    )
    
    globals()[d]['file_year'] = d.split('_')[-1]
    globals()[d] = globals()[d][['system_code','school_code','system_name','school_name','site_type','nces_id','is_charter', 'street', 'city', 'state', 'zip_5','county','file_year', 'opened_date']]
    
df_directory = pd.concat([df_directory_2022,
                          df_directory_2021,
                          df_directory_2020,
                          df_directory_2019,
                          df_directory_2018,
                          df_directory_2017,
                          df_directory_2016,
                          df_directory_2015])

df_directory.to_csv('../raw_data_files_clean/merged_directory.csv', encoding='utf-8', index=False)

In [17]:
df_directory.loc[df_directory['system_name']=='i3 Academy']

,system_code,school_code,system_name,school_name,site_type,nces_id,is_charter,street,city,state,zip_5,county,file_year,opened_date
1601,805,0,i3 Academy,i3 Academy,Central Office,100204.0,NaN,55th Place 1st Avenue South,Birmingham,AL,35212,Jefferson,2022,7/1/20
1602,805,1,i3 Academy,i3 Academy,Public Board of Education,10020402501.0,NaN,55th Place 1st Avenue South,Birmingham,AL,35212-,Jefferson,2022,7/1/20
1603,805,10,i3 Academy,i3 Academy - Elementary School,Public Charter School Startup,10020402476.0,True,55th Place 1st Avenue South,Birmingham,AL,35212,Jefferson,2022,7/1/20
1604,805,15,i3 Academy,i3 Academy - Middle School,Public Charter School Startup,10020402510.0,True,7901 1st Avenue North,Birmingham,AL,35206-,Jefferson,2022,7/1/21
1605,805,4999,i3 Academy,Spec Ed Private Services (Child Count Only),Special Education - Private Services,NaN,False,55th Place 1st Avenue South,Birmingham,AL,35212,Jefferson,2022,4/2/21
1606,805,7001,i3 Academy,Summer School ñ Elementary,Summer School,NaN,Summer School,55th Place 1st Avenue South,Birmingham,AL,35212,Jefferson,2022,11/19/20
1607,805,7002,i3 Academy,Summer School ñ Middle,Summer School,NaN,Summer School,55th Place 1st Avenue South,Birmingham,AL,35212,Jefferson,2022,11/19/20
1608,805,7003,i3 Academy,Summer School ñ Secondary,Summer School,NaN,Summer School,55th Place 1st Avenue South,Birmingham,AL,35212,Jefferson,2022,11/19/20
1609,805,7004,i3 Academy,Virtual Program,Virtual Program,NaN,Virtual Program,55th Place 1st Avenue South,Birmingham,AL,35212,Jefferson,2022,11/19/20
1599,805,0,i3 Academy,i3 Academy,Central Office,100204.0,NaN,55th Place 1st Avenue South,Birmingham,AL,35212-,Jefferson,2021,7/1/20


In [18]:
df_assessment_2015 = pd.concat([df_assessment_math_2015,df_assessment_ela_2015])
df_assessment_2016 = pd.concat([df_assessment_math_2016,df_assessment_ela_2016])
df_assessment_2017 = pd.concat([df_assessment_math_2017,df_assessment_ela_2017])
df_assessment_2018 = pd.concat([df_assessment_math_2018,df_assessment_ela_2018])
df_assessment_2019 = pd.concat([df_assessment_math_2019,df_assessment_ela_2019])
df_assessment_2021 = pd.concat([df_assessment_math_2021,df_assessment_ela_2021])
df_assessment_2022 = pd.concat([df_assessment_math_2022,df_assessment_ela_2022])


In [19]:
df_dir_2022_filtered = df_directory_2022.loc[(df_directory_2022['system_code'].str.isnumeric())].copy()
df_dir_2022_filtered['system_code'] = df_dir_2022_filtered['system_code'].astype('int64')
df_dir_2022_filtered['school_code'] = df_dir_2022_filtered['school_code'].astype('int64')

In [20]:
df_dir_2022_filtered.loc[(df_dir_2022_filtered['system_code']==803)]

,system_code,school_code,system_name,school_name,site_type,nces_id,is_charter,street,city,state,zip_5,county,file_year,opened_date
1903,803,0,Legacy Prep,Legacy Prep,Central Office,1.002010e+05,NaN,1500 Daniel Payne Drive,Birmingham,AL,35214-,Washington,2022,7/1/19
1904,803,0,Legacy Prep,Legacy Prep,Central Office,1.002010e+05,NaN,1500 Daniel Payne Drive,Birmingham,AL,35214-,Washington,2022,7/1/19
1905,803,0,Legacy Prep,Legacy Prep,Central Office,1.002010e+05,NaN,1500 Daniel Payne Drive,Birmingham,AL,35214-,Washington,2022,7/1/19
1906,803,0,Legacy Prep,Legacy Prep,Central Office,1.002010e+05,NaN,1500 Daniel Payne Drive,Birmingham,AL,35214-,Washington,2022,7/1/19
1907,803,10,Legacy Prep,Legacy Prep,Public Charter School Startup,1.002010e+10,True,1500 Daniel Payne Drive,Birmingham,AL,35214-,Washington,2022,7/1/19
1908,803,1,Legacy Prep,Legacy Prep,Public Board of Education,NaN,NaN,1500 Daniel Payne Drive,Birmingham,AL,35214-,Jefferson,2022,7/1/19
1909,803,4999,Legacy Prep,Spec Ed Private Services (Child Count Only),Special Education - Private Services,NaN,False,1500 Daniel Payne Drive,Birmingham,AL,35214-,Washington,2022,4/2/21
1910,803,7001,Legacy Prep,Summer School ñ Elementary,Summer School,NaN,Summer School,1500 Daniel Payne Drive,Birmingham,AL,35214-,Washington,2022,11/19/20
1911,803,7002,Legacy Prep,Summer School ñ Middle,Summer School,NaN,Summer School,1500 Daniel Payne Drive,Birmingham,AL,35214-,Washington,2022,11/19/20
1912,803,7003,Legacy Prep,Summer School ñ Secondary,Summer School,NaN,Summer School,1500 Daniel Payne Drive,Birmingham,AL,35214-,Washington,2022,11/19/20


In [21]:
# df_assessment_2022_raw = pd.merge(df_assessment_2022, df_dir_2022_filtered, on=['system_code', 'school_code'], how='left')
# df_assessment_2022_raw

In [22]:
# df_assessment_2022_raw = pd.merge(df_assessment_2022_raw, df_district_charter_mapping, on=['system_code','school_code'], how='left')
# df_assessment_2022_raw['updated_district'] =  df_assessment_2022_raw['updated_district'].fillna(df_assessment_2022_raw['system_name_x'])
# df_assessment_2022_raw['is_charter_district'] = np.where(df_assessment_2022_raw['updated_district']=='Birmingham City','True',
#                                                np.where(df_assessment_2022_raw['updated_district']=='Homewood City','True',
#                                                         np.where(df_assessment_2022_raw['updated_district']=='Mobile County','True',
#                                                                  np.where(df_assessment_2022_raw['updated_district']=='Montgomery County','True',
#                                                                           np.where(df_assessment_2022_raw['updated_district']=='Perry County','True',
#                                                                                    np.where(df_assessment_2022_raw['updated_district']=='Sumter County','True','False'))))))
# df_assessment_2022_raw.to_csv('../raw_data_files_clean/merged_assessment_2022.csv', encoding='utf-8', index=False)

In [23]:
# df_assessment_2022.loc[(df_assessment_2022['system_code']>=800) 
#                        & (df_assessment_2022['grade']=='ALL')
#                        & (df_assessment_2022['gender']=='All Gender')
#                        & (df_assessment_2022['race']=='All Race')
#                        & (df_assessment_2022['ethnicity']=='All Ethnicity')
#                        & (df_assessment_2022['subpopulation']=='All SubPopulation')
#                       ]


In [24]:
# df_assessmentparticipation_2022['total_students_tested_clean'] = np.where(df_assessmentparticipation_2022['total_students_tested'] == '*', 
#                                                                      np.NaN, 
#                                                                      df_assessmentparticipation_2022['total_students_tested']).astype('float64')

In [25]:
# df_assessmentparticipation_2022['total_tested'] = round(df_assessmentparticipation_2022['total_students_tested_clean']/(df_assessmentparticipation_2022['percent_of_all_students_tested']/100),0)


# ## NEED TO MANUALLY ADD THE LEAD ACADEMY ES VALUE BASED ON TOTAL LEAD ACADEMY - LEAD ACADEMY MS TOTAL
# ## 'tested'=64 because 271-207 

# lead_es_math_index = df_assessmentparticipation_2022.loc[(df_assessmentparticipation_2022['system_code']==802) 
#                                     & (df_assessmentparticipation_2022['school_code']==10)
#                                     & (df_assessmentparticipation_2022['grade']=='ALL')
#                                     & (df_assessmentparticipation_2022['subject']=='Math')
#                                     & (df_assessmentparticipation_2022['test_type']=='Regular Assessment')].index[0]

# lead_es_ela_index = df_assessmentparticipation_2022.loc[(df_assessmentparticipation_2022['system_code']==802) 
#                                     & (df_assessmentparticipation_2022['school_code']==10)
#                                     & (df_assessmentparticipation_2022['grade']=='ALL')
#                                     & (df_assessmentparticipation_2022['subject']=='ELA')
#                                     & (df_assessmentparticipation_2022['test_type']=='Regular Assessment')].index[0]

# df_assessmentparticipation_2022.at[lead_es_math_index, 'total_tested']=64
# df_assessmentparticipation_2022.at[lead_es_ela_index, 'total_tested']=64

# df_total_tested_2022 = df_assessmentparticipation_2022[['system_code',
#                                                          'system_name',
#                                                          'school_code',
#                                                          'school_name',
#                                                          'subject',
#                                                          'grade',
#                                                          'total_tested']].drop_duplicates().dropna()

# df_total_tested_2022_avg = (
#     df_total_tested_2022
#     .groupby(['system_code','system_name','school_code','school_name','subject', 'grade'])
#     .mean()
#     .reset_index()
# )

# df_total_tested_2022_avg['tested'] = round(df_total_tested_2022_avg['total_tested'],0)

# df_assessment_2022 = df_assessment_2022.merge(df_total_tested_2022_avg[['system_code',
#                                                                         'school_code','subject', 'grade','tested']],
#                            left_on=['system_code','school_code','subject', 'grade'], 
#                            right_on=['system_code','school_code','subject', 'grade'], 
#                            how='left')



In [26]:
# df_assessment_2022

In [27]:
for i in ['df_assessment_2022','df_assessment_2021','df_assessment_2019','df_assessment_2018','df_assessment_2017']: 
    # globals()[i] = globals()[i].rename(columns={'subpopulation':'sub_population', 'percent_level1':'level_1_%', 'percent_level2':'level_2_%', 'percent_level1':'level_3_%', 'percent_level1':'level_4_%'})
    globals()[i] = globals()[i].loc[(globals()[i]['gender']=='All Gender')
                                                         # & (globals()[i]['race']=='All Race')
                                                         # & (globals()[i]['ethnicity']=='All Ethnicity')
                                                         # & (globals()[i]['sub_population']=='All SubPopulation')
                                                         # & (globals()[i]['grade']=='All Grades')
                                                        ].copy()
    year = i.split('_')[-1]
    
    globals()[i]['pct_not_proficient'] = globals()[i]['level_1_%'].replace('*',np.NaN).replace('~',np.NaN).astype('float')/100 + globals()[i]['level_2_%'].replace('*',np.NaN).replace('~',np.NaN).astype('float')/100 
    globals()[i]['proficient_rate_derived'] = (1 - globals()[i]['pct_not_proficient']) * 100   
    globals()[i]['proficient_rate'] = globals()[i]['proficient_rate'].replace('*',np.NaN).replace('~',np.NaN)
    globals()[i]['proficient_rate'].fillna(value=globals()[i]['proficient_rate_derived'], inplace=True)
    
    globals()[i]['tested_derived'] = (globals()[i]['level_1'].replace('*',np.NaN).replace('~',np.NaN).astype('float') + globals()[i]['level_2'].replace('*',np.NaN).replace('~',np.NaN).astype('float')) / globals()[i]['pct_not_proficient']
    globals()[i]['tested'] = globals()[i]['tested'].replace('*',np.NaN).replace('~',np.NaN)
    globals()[i]['tested'].fillna(value=globals()[i]['tested_derived'], inplace=True)
    
    
    globals()[i]['proficient_derived'] = globals()[i]['tested_derived'] - (globals()[i]['level_1'].replace('*',np.NaN).replace('~',np.NaN).astype('float') + globals()[i]['level_2'].replace('*',np.NaN).replace('~',np.NaN).astype('float'))
    globals()[i]['proficient'] = globals()[i]['proficient'].replace('*',np.NaN).replace('~',np.NaN)
    globals()[i]['proficient'].fillna(value=globals()[i]['proficient_derived'], inplace=True)
    
    
    globals()[i] = globals()[i].rename(columns={'proficient_rate':'percent_proficient'})
    globals()[i] = globals()[i].replace('*',np.NaN).replace('~',np.NaN)
    globals()[i]['subject'] = globals()[i]['subject'].replace('Reading','ELA')
    for x in ['tested','proficient','percent_proficient']:
        globals()[i][year+'_'+x] = globals()[i][x].astype('float')
    globals()[i][year+'_tested'] = globals()[i][year+'_tested'].combine_first(globals()[i][year+'_proficient'] / (globals()[i][year+'_percent_proficient']/100))
    globals()[i+'_all_students_pivot'] = globals()[i].pivot_table(index=['system_name','school_name'], columns='subject', values=[year+'_percent_proficient',year+'_tested', year+'_proficient']).reset_index()
    globals()[i+'_all_students_pivot'].columns = globals()[i+'_all_students_pivot'].columns.map('_'.join).str.lower().str.strip('_')

In [35]:
df_assessment_2021

,year,system_name,school_name,subject,grade,gender,race,ethnicity,sub_population,enrolled,tested,proficient,participation_rate,percent_proficient,level_1,level_2,level_3,level_4,level_1_%,level_2_%,level_3_%,level_4_%,pct_not_proficient,proficient_rate_derived,tested_derived,proficient_derived,2021_tested,2021_proficient,2021_percent_proficient
0,2021,Acceleration Day and Evening Academy,Acceleration Day and Evening Academy,Math,All Grades,All Gender,All Race,All Ethnicity,All SubPopulation,89.47,53.00,1.001733,59.24,1.89,40,12,NaN,NaN,75.47,22.64,NaN,0.00,0.9811,1.89,53.001733,1.001733,53.0,1.001733,1.89
1,2021,Acceleration Day and Evening Academy,Acceleration Day and Evening Academy,Math,All Grades,All Gender,All Race,All Ethnicity,Students with Disabilities,15.13,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,0.00
2,2021,Acceleration Day and Evening Academy,Acceleration Day and Evening Academy,Math,All Grades,All Gender,All Race,All Ethnicity,General Education Students,74.34,43.00,0.997543,57.84,2.32,30,12,NaN,NaN,69.77,27.91,NaN,0.00,0.9768,2.32,42.997543,0.997543,43.0,0.997543,2.32
3,2021,Acceleration Day and Evening Academy,Acceleration Day and Evening Academy,Math,All Grades,All Gender,All Race,All Ethnicity,Economically Disadvantaged,77.00,43.00,NaN,55.84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,43.0,NaN,NaN
4,2021,Acceleration Day and Evening Academy,Acceleration Day and Evening Academy,Math,All Grades,All Gender,All Race,Hispanic/Latino,All SubPopulation,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,0.00,NaN,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975917,2021,Mobile County,H.L. Sonny Callahan School for the Deaf and Blind,ELA,Grade 05,All Gender,White,All Ethnicity,Military Family,NaN,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN,0.00,0.00,NaN,0.00,0.0000,100.00,NaN,NaN,NaN,NaN,100.00
975918,2021,Mobile County,H.L. Sonny Callahan School for the Deaf and Blind,ELA,Grade 05,All Gender,White,Other Ethnicity,All SubPopulation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
975919,2021,Mobile County,H.L. Sonny Callahan School for the Deaf and Blind,ELA,Grade 05,All Gender,White,Other Ethnicity,Students with Disabilities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
975920,2021,Mobile County,H.L. Sonny Callahan School for the Deaf and Blind,ELA,Grade 05,All Gender,White,Other Ethnicity,Economically Disadvantaged,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,0.00


In [29]:
# # globals()[i] = globals()[i].rename(columns={'subpopulation':'sub_population', 'percent_level1':'level_1_%', 'percent_level2':'level_2_%', 'percent_level1':'level_3_%', 'percent_level1':'level_4_%'})
# df_assessment_2022 = df_assessment_2022.loc[(df_assessment_2022['gender']=='All Gender')
#                                                      & (df_assessment_2022['race']=='All Race')
#                                                      & (df_assessment_2022['ethnicity']=='All Ethnicity')
#                                                      & (df_assessment_2022['subpopulation']=='All SubPopulation')
#                                                      & ((df_assessment_2022['grade']=='All Grades') | (df_assessment_2022['grade']=='ALL'))
#                                                     ].copy()

# df_assessment_2022 = df_assessment_2022.rename(columns={'percent_level1':'level_1_%',
#                                     'percent_level2':'level_2_%',
#                                     'percent_level3':'level_3_%',
#                                     'percent_level4':'level_4_%',
#                                     'percent_proficient':'proficient_rate'})

# df_assessment_2022['pct_not_proficient'] = df_assessment_2022['level_1_%'].replace('*',np.NaN).replace('~',np.NaN).astype('float')/100 + df_assessment_2022['level_2_%'].replace('*',np.NaN).replace('~',np.NaN).astype('float')/100 
# df_assessment_2022['proficient_rate_derived'] = (1 - df_assessment_2022['pct_not_proficient']) * 100   
# df_assessment_2022['proficient_rate'] = df_assessment_2022['proficient_rate'].replace('*',np.NaN).replace('~',np.NaN)
# df_assessment_2022['proficient_rate'].fillna(value=df_assessment_2022['proficient_rate_derived'], inplace=True)

# df_assessment_2022['level_1'] = (df_assessment_2022['level_1_%'].replace('*',np.NaN).replace('~',np.NaN).astype('float')/100) * df_assessment_2022['tested']
# df_assessment_2022['level_2'] = (df_assessment_2022['level_2_%'].replace('*',np.NaN).replace('~',np.NaN).astype('float')/100) * df_assessment_2022['tested']
# df_assessment_2022['level_3'] = (df_assessment_2022['level_3_%'].replace('*',np.NaN).replace('~',np.NaN).astype('float')/100) * df_assessment_2022['tested']
# df_assessment_2022['level_4'] = (df_assessment_2022['level_4_%'].replace('*',np.NaN).replace('~',np.NaN).astype('float')/100) * df_assessment_2022['tested']

# df_assessment_2022['tested_derived'] = (df_assessment_2022['level_1'].replace('*',np.NaN).replace('~',np.NaN).astype('float') + df_assessment_2022['level_2'].replace('*',np.NaN).replace('~',np.NaN).astype('float')) / df_assessment_2022['pct_not_proficient']
# df_assessment_2022['tested'] = df_assessment_2022['tested'].replace('*',np.NaN).replace('~',np.NaN)
# df_assessment_2022['tested'].fillna(value=df_assessment_2022['tested_derived'], inplace=True)

# df_assessment_2022['proficient'] = df_assessment_2022['level_3']+df_assessment_2022['level_4']
# df_assessment_2022['proficient_derived'] = df_assessment_2022['tested_derived'] - (df_assessment_2022['level_1'].replace('*',np.NaN).replace('~',np.NaN).astype('float') + df_assessment_2022['level_2'].replace('*',np.NaN).replace('~',np.NaN).astype('float'))
# df_assessment_2022['proficient'] = df_assessment_2022['proficient'].replace('*',np.NaN).replace('~',np.NaN)
# df_assessment_2022['proficient'].fillna(value=df_assessment_2022['proficient_derived'], inplace=True)


# df_assessment_2022 = df_assessment_2022.rename(columns={'proficient_rate':'percent_proficient'})
# df_assessment_2022 = df_assessment_2022.replace('*',np.NaN).replace('~',np.NaN)
# df_assessment_2022['subject'] = df_assessment_2022['subject'].replace('Reading','ELA')
# for x in ['tested','proficient','percent_proficient']:
#      df_assessment_2022['2022_'+x] = df_assessment_2022[x].astype('float')
# df_assessment_2022['2022_tested'] = df_assessment_2022['2022_tested'].combine_first(df_assessment_2022['2022_proficient'] / (df_assessment_2022['2022_percent_proficient']/100))


# df_assessment_2022

In [61]:
# df_assessment_2022['proficient'] = (df_assessment_2022['percent_proficient'].replace('*',np.NaN).astype('float')/100) * df_assessment_2022['tested'].astype('float')

l = [
    'df_assessment_2022',
    'df_assessment_2021',
    'df_assessment_2019',
    'df_assessment_2018',
    'df_assessment_2017',
    'df_assessment_2016',
    'df_assessment_2015'
]

c = ['system_name', 'school_name', 'subject', 'grade','tested', 'proficient','race','ethnicity','sub_population','grade',
       'participation_rate', 'proficient_rate'
    ]
df_to_change_and_merge(l, c)

In [ ]:
df_assessment_2022.loc[(df_assessment_2022['system_name']=='i3 Academy') 
                  # & 
                       # (df_assessment['grade']=='ALL') & 
                       # (df_assessment['gender']=='All Gender') & 
                       # (df_assessment['race']=='All Race') & 
                       # (df_assessment['ethnicity']=='All Ethnicity') & 
                       # (df_assessment['subpopulation']=='All SubPopulation')
                      ]

## Building a flat file

In [ ]:
df_assessment_2021.rename(columns={'percent_proficient':'proficient_rate'})
df_assessment_2021

In [ ]:
for i in ['df_assessment_2021','df_assessment_2019','df_assessment_2018','df_assessment_2017']: 
    globals()[i] = globals()[i].rename(columns={'percent_proficient':'proficient_rate'})
    globals()[i+'_all_students'] = globals()[i].loc[(globals()[i]['gender']=='All Gender')
                                                         & (globals()[i]['race']=='All Race')
                                                         & (globals()[i]['ethnicity']=='All Ethnicity')
                                                         & (globals()[i]['sub_population']=='All SubPopulation')
                                                         & (globals()[i]['grade']=='All Grades')
                                                        ].copy()
    year = i.split('_')[-1]
    
    globals()[i+'_all_students']['pct_not_proficient'] = globals()[i+'_all_students']['level_1_%'].replace('*',np.NaN).replace('~',np.NaN).astype('float')/100 + globals()[i+'_all_students']['level_2_%'].replace('*',np.NaN).replace('~',np.NaN).astype('float')/100 
    globals()[i+'_all_students']['proficient_rate_derived'] = (1 - globals()[i+'_all_students']['pct_not_proficient']) * 100   
    globals()[i+'_all_students']['proficient_rate'] = globals()[i+'_all_students']['proficient_rate'].replace('*',np.NaN).replace('~',np.NaN)
    globals()[i+'_all_students']['proficient_rate'].fillna(value=globals()[i+'_all_students']['proficient_rate_derived'], inplace=True)
    
    globals()[i+'_all_students']['tested_derived'] = (globals()[i+'_all_students']['level_1'].replace('*',np.NaN).replace('~',np.NaN).astype('float') + globals()[i+'_all_students']['level_2'].replace('*',np.NaN).replace('~',np.NaN).astype('float')) / globals()[i+'_all_students']['pct_not_proficient']
    globals()[i+'_all_students']['tested'] = globals()[i+'_all_students']['tested'].replace('*',np.NaN).replace('~',np.NaN)
    globals()[i+'_all_students']['tested'].fillna(value=globals()[i+'_all_students']['tested_derived'], inplace=True)
    
    
    globals()[i+'_all_students']['proficient_derived'] = globals()[i+'_all_students']['tested_derived'] - (globals()[i+'_all_students']['level_1'].replace('*',np.NaN).replace('~',np.NaN).astype('float') + globals()[i+'_all_students']['level_2'].replace('*',np.NaN).replace('~',np.NaN).astype('float'))
    globals()[i+'_all_students']['proficient'] = globals()[i+'_all_students']['proficient'].replace('*',np.NaN).replace('~',np.NaN)
    globals()[i+'_all_students']['proficient'].fillna(value=globals()[i+'_all_students']['proficient_derived'], inplace=True)
    
    
    globals()[i+'_all_students'] = globals()[i+'_all_students'].rename(columns={'proficient_rate':'percent_proficient'})
    globals()[i+'_all_students'] = globals()[i+'_all_students'].replace('*',np.NaN).replace('~',np.NaN)
    globals()[i+'_all_students']['subject'] = globals()[i+'_all_students']['subject'].replace('Reading','ELA')
    for x in ['tested','proficient','percent_proficient']:
        globals()[i+'_all_students'][year+'_'+x] = globals()[i+'_all_students'][x].astype('float')
    globals()[i+'_all_students'][year+'_tested'] = globals()[i+'_all_students'][year+'_tested'].combine_first(globals()[i+'_all_students'][year+'_proficient'] / (globals()[i+'_all_students'][year+'_percent_proficient']/100))
    globals()[i+'_all_students_pivot'] = globals()[i+'_all_students'].pivot_table(index=['system_name','school_name'], columns='subject', values=[year+'_percent_proficient',year+'_tested', year+'_proficient']).reset_index()
    globals()[i+'_all_students_pivot'].columns = globals()[i+'_all_students_pivot'].columns.map('_'.join).str.lower().str.strip('_')

In [ ]:
df_assessment_2021.loc[df_assessment_2021['system_name']=='Legacy Prep'].drop_duplicates()

In [ ]:
df_assessment_2021.loc[
    (df_assessment_2021['level_3_%']=='*') 
                       & (df_assessment_2021['level_4_%']=='*')
                      & (df_assessment_2021['grade']=='All Grades')
                      & (df_assessment_2021['gender']=='All Gender')
                      & (df_assessment_2021['race']=='All Race')
                      & (df_assessment_2021['ethnicity']=='All Ethnicity')
                      & (df_assessment_2021['sub_population']=='All SubPopulation')
                      ]

df_assessment_2021_all_students_pivot.loc[df_assessment_2021_all_students_pivot['system_name']=='Legacy Prep']

In [ ]:
df_assessment_2022_all_students = df_assessment_2022.loc[(df_assessment_2022['gender']=='All Gender')
                                                         & (df_assessment_2022['race']=='All Race')
                                                         & (df_assessment_2022['ethnicity']=='All Ethnicity')
                                                         & (df_assessment_2022['subpopulation']=='All SubPopulation')
                                                         & (df_assessment_2022['grade']=='ALL')
                                                         & (((df_assessment_2022['school_code']!=0)
                                                         & (df_assessment_2022['school_code']!=1)) | (df_assessment_2022['system_name']=='Alabama State Department of Education'))
                                                        ]

df_assessment_2022_all_students = df_assessment_2022_all_students[['system_code', 'system_name', 'school_code', 'school_name', 'subject', 'percent_proficient', 'tested']]

df_assessment_2022_all_students['2022_percent_proficient'] = df_assessment_2022_all_students['percent_proficient'].replace('*',np.NaN).astype('float')
df_assessment_2022_all_students['2022_tested'] = df_assessment_2022_all_students['tested'].replace('*',np.NaN).astype('float')
df_assessment_2022_all_students['2022_proficient'] = df_assessment_2022_all_students['2022_tested'] * (df_assessment_2022_all_students['2022_percent_proficient']/100)

df_assessment_2022_all_students_pivot = df_assessment_2022_all_students.pivot_table(index=['system_code','system_name','school_code','school_name'], columns='subject', values=['2022_percent_proficient','2022_tested','2022_proficient']).reset_index()
df_assessment_2022_all_students_pivot.columns = df_assessment_2022_all_students_pivot.columns.map('_'.join).str.lower().str.strip('_')

df_assessment_2022_all_students_pivot

In [ ]:
df_directory_2022_updated = df_directory.loc[((df_directory['nces_id'].notnull()) | (df_directory['school_name']=='Alabama State Department of Education')) & (df_directory['file_year']=='2022')].copy()
df_directory_2022_updated['system_code'] = df_directory_2022_updated['system_code'].replace('999',0)
df_directory_2022_updated['system_code'] = df_directory_2022_updated['system_code'].astype('int64')
df_directory_2022_updated['school_code'] = df_directory_2022_updated['school_code'].astype('int64')

df_assessment_2022_all_students_pivot = df_assessment_2022_all_students_pivot.merge(df_directory_2022_updated[['system_code','school_code','site_type','county','is_charter']], 
                                                                        left_on=['system_code','school_code'],
                                                                        right_on=['system_code','school_code'],
                                                                        how='left')

In [ ]:
df_assessment_2022_all_students_pivot

#### ADM

In [ ]:
df_assessment_2022

In [ ]:
for a, n in zip([df_adm_2022,df_adm_2021,df_adm_2020], 
                ['df_adm_school_2022','df_adm_school_2021','df_adm_school_2020']):
    s = a.copy()
    s['fill_down'] = s['system_name'].ffill()
    s[['code','name']] = s['fill_down'].str.split(n=1, expand=True)
    s['system_code'] = np.where(s['code'].str.len()==4, np.NaN, s['code'])
    s['system_code'] = s['system_code'].ffill()
    s['school_code'] = np.where(s['code'].str.len()==3, '0000', s['code'])
    s['file_source'] = n
    s['file_year'] = n.split('_')[-1]
    s = s.replace('State','0000').replace(',','', regex=True)
    s['system_code'] = s['system_code'].astype('int64')
    s['system_code'] = s['system_code'].astype('string')
    s['school_code'] = s['school_code'].astype('int64')
    s['k12'] = s['k12'].astype('float')

    vars()[n] = s
    
   
    
df_merged_adm = pd.concat([df_adm_school_2022[['system_code','school_code','k12','file_source','file_year']],
          df_adm_school_2021[['system_code','school_code','k12','file_source','file_year']],
          df_adm_school_2020[['system_code','school_code','k12','file_source','file_year']]])

df_merged_adm = df_merged_adm.merge(df_directory, left_on=['system_code','school_code','file_year'], right_on=['system_code','school_code','file_year'], how='left')
df_merged_adm = df_merged_adm.drop_duplicates()
df_merged_adm.to_csv('../raw_data_files_clean/merged_adm.csv', encoding='utf-8', index=False)

#### Demographics

In [ ]:
df_frl_2022

In [ ]:
df_frl_2022['system_fill_down'] = df_frl_2022['system_name'].ffill()
df_frl_2022_schools = df_frl_2022.loc[df_frl_2022['school_name'].notnull()].copy()
df_frl_2022_schools[['system_code','system_name']] = df_frl_2022_schools['system_fill_down'].str.split(n=1, expand=True)
df_frl_2022_schools[['school_code','school_name']] = df_frl_2022_schools['school_name'].str.split(n=1, expand=True)
df_frl_2022_schools = df_frl_2022_schools[['system_code', 'school_code', '%_free_reduced_', 'enrollment_']]
df_frl_2022_schools['system_code'] = df_frl_2022_schools['system_code'].astype('int64')
df_frl_2022_schools['school_code'] = df_frl_2022_schools['school_code'].astype('int64')
df_frl_2022_schools.columns = ['system_code','school_code','frl_pct_2022','total_frl_enrollment_2022']
df_frl_2022_schools

In [ ]:
state_frl = pd.DataFrame({'system_code':[0], 'school_code':[0], 'frl_pct_2022':['44.201%'], 'total_frl_enrollment_2022':[725900]})
df_frl_2022_schools = pd.concat([df_frl_2022_schools, state_frl])
df_frl_2022_schools

In [ ]:
# df_attendance_school_2022_pivot = df_attendance_school_2022.pivot_table(index=['system_code','school_code'], columns='demographics_group', values='k12', aggfunc=np.sum).reset_index()
# df_attendance_school_2022_pivot = df_attendance_school_2022_pivot.replace(np.NaN, 0)
# df_attendance_school_2022_pivot['total_sum_demo'] = (df_attendance_school_2022_pivot['american_indian_alaska_native'] + 
#                                                        df_attendance_school_2022_pivot['asian'] + 
#                                                        df_attendance_school_2022_pivot['black_or_african_american'] + 
#                                                        df_attendance_school_2022_pivot['hispanic_latino'] +
#                                                        df_attendance_school_2022_pivot['native_hawaiian_pacific_islander'] + 
#                                                        df_attendance_school_2022_pivot['not_reported'] +
#                                                        df_attendance_school_2022_pivot['two_or_more_races'] + 
#                                                        df_attendance_school_2022_pivot['white'])

# df_attendance_school_2022_pivot['missing_2022'] = df_attendance_school_2022_pivot['all_students'] - df_attendance_school_2022_pivot['total_sum_demo']
# for c in ['american_indian_alaska_native', 'asian',
#        'black_or_african_american', 'hispanic_latino',
#        'native_hawaiian_pacific_islander', 'not_reported',
#        'two_or_more_races', 'white',
#        'total_sum_demo', 'missing_2022']:
#     df_attendance_school_2022_pivot[c + '_pct_2022'] = df_attendance_school_2022_pivot[c]/df_attendance_school_2022_pivot['all_students']
#     df_attendance_school_2022_pivot = df_attendance_school_2022_pivot.rename(columns={c:c+'_2022'})
    
# df_attendance_school_2022_pivot = df_attendance_school_2022_pivot.rename(columns={'all_students':'all_students_2022'})
# df_attendance_school_2022_pivot = df_attendance_school_2022_pivot.drop(columns='unknown')
# df_attendance_school_2022_pivot['system_code'] = df_attendance_school_2022_pivot['system_code'].astype('int64')
# df_attendance_school_2022_pivot['school_code'] = df_attendance_school_2022_pivot['school_code'].astype('int64')
# df_attendance_school_2022_pivot

In [ ]:
df_demographics_2022[df_demographics_2022['system_name'] == df_demographics_2022['school_name']]

In [ ]:
df_demographics_2022_hispanic = df_demographics_2022.loc[(df_demographics_2022['ethnicity']=='Hispanic/Latino') 
                                                         & (df_demographics_2022['sub_population']=='All SubPopulation')
                                                         & (df_demographics_2022['grade']=='All Grades')
                                                         & (df_demographics_2022['gender']=='All Gender')
                                                        ].drop_duplicates().copy()
df_demographics_2022_non_hispanic = df_demographics_2022.loc[(df_demographics_2022['ethnicity']=='Other Ethnicity') 
                                                             & (df_demographics_2022['sub_population']=='All SubPopulation')
                                                         & (df_demographics_2022['grade']=='All Grades')
                                                         & (df_demographics_2022['gender']=='All Gender')].drop_duplicates().copy()
df_demographics_2022_all = df_demographics_2022.loc[(df_demographics_2022['ethnicity']=='All Ethnicity') 
                                                # & (df_demographics_2022['sub_population']=='All SubPopulation')
                                                         & (df_demographics_2022['grade']=='All Grades')
                                                         & (df_demographics_2022['gender']=='All Gender')].drop_duplicates().copy()
df_demographics_2022_hispanic


In [ ]:
df_demographics_sub_pop_2022 = df_demographics_2022_all[['year','system_name','school_name','sub_population','total_student_count']].replace('*',np.NaN).copy()
df_demographics_sub_pop_2022['total_student_count'] = df_demographics_sub_pop_2022['total_student_count'].astype('float')
df_demographics_sub_pop_2022 = df_demographics_sub_pop_2022.pivot_table(index=['year','system_name', 'school_name'], columns='sub_population', values='total_student_count').reset_index()

df_demographics_sub_pop_2022.columns = ['year','system_name','school_name','total_enrollment_2022','econ_disadv','foster','gen_ed','homeless','migrant','military','swd','el']  
for c in ['econ_disadv','foster','gen_ed','homeless','migrant','military','swd','el']: 
    df_demographics_sub_pop_2022[c+'_pct_2022'] = df_demographics_sub_pop_2022[c] / df_demographics_sub_pop_2022['total_enrollment_2022']
df_demographics_sub_pop_2022

In [ ]:
df_demographics_2022_hispanic[df_demographics_2022_hispanic['system_name']=='i3 Academy']

In [ ]:
df_demographics_2022_ethnicity = df_demographics_2022_non_hispanic[['system_name','school_name',
                                    'asian', 'black_or_african_american', 
                                    'american_indian_/_alaska_native', 'native_hawaiian_/_pacific_islander',
                                    'white', 'two_or_more_races']].merge(df_demographics_2022_hispanic[['system_name','school_name','total_student_count']], 
                                              on=['system_name','school_name'])

df_demographics_2022_ethnicity.columns = ['system_name','school_name',
                                    'asian', 'black_or_african_american', 
                                    'american_indian_alaska_native', 'native_hawaiian_pacific_islander',
                                    'white', 'two_or_more_races', 'hispanic_latino']

df_demographics_pct_2022 = df_demographics_2022_ethnicity.merge(df_demographics_sub_pop_2022,left_on=['system_name','school_name'],right_on=['system_name','school_name'],how='outer').drop_duplicates()

In [ ]:
df_demographics_2022_ethnicity[df_demographics_2022_ethnicity['system_name']=='i3 Academy']

In [ ]:
field = ['asian',  
         'black_or_african_american', 
         'american_indian_alaska_native',
         'native_hawaiian_pacific_islander',
         'white','two_or_more_races', 
         'hispanic_latino']
for z in field: 
    df_demographics_pct_2022[z+'_pct_2022'] = df_demographics_pct_2022[z].replace('*',np.NaN).astype('float')/df_demographics_pct_2022['total_enrollment_2022'].replace('*',np.NaN).astype('float')
df_demographics_pct_2022
    
df_demographics_pct_2022['sum_pct_2022'] = (
    df_demographics_pct_2022['asian_pct_2022']
    +df_demographics_pct_2022['black_or_african_american_pct_2022']
    +df_demographics_pct_2022['american_indian_alaska_native_pct_2022']
    +df_demographics_pct_2022['native_hawaiian_pacific_islander_pct_2022']
    +df_demographics_pct_2022['white_pct_2022']
    +df_demographics_pct_2022['two_or_more_races_pct_2022']
    +df_demographics_pct_2022['hispanic_latino_pct_2022']
)


In [ ]:
df_demographics_pct_2022[df_demographics_pct_2022['system_name']==df_demographics_pct_2022['school_name']]
df_demographics_pct_2022.to_csv('../data_for_analysis/2022_demog.csv', encoding='utf-8', index=False)

In [ ]:
df_flat_file = df_assessment_2022_all_students_pivot.merge(df_demographics_pct_2022,
                                      left_on=['system_name','school_name'],
                                      right_on=['system_name','school_name'],
                                      how='left')
df_flat_file = df_flat_file.merge(df_district_charter_mapping, on=['system_code','school_code'], how='left')
df_flat_file['updated_district'].fillna(value=df_flat_file['system_name'], inplace=True)
df_flat_file['updated_district_code'].fillna(value=df_flat_file['system_code'], inplace=True)
df_flat_file = df_flat_file.merge(df_frl_2022_schools, on=['system_code','school_code'], how='left')
df_flat_file

In [ ]:
# df_flat_file = df_flat_file.merge(df_attendance_school_2022_pivot, on=['system_code','school_code'], how='left')

In [ ]:
df_flat_file.loc[df_flat_file['school_name']=='Alabama State Department of Education']

In [ ]:
df_flat_file['black_or_hispanic_pct_2022'] = df_flat_file['hispanic_latino_pct_2022'] + df_flat_file['black_or_african_american_pct_2022']
df_flat_file['is_charter_district'] = np.where(df_flat_file['updated_district']=='Birmingham City','True',
                                               np.where(df_flat_file['updated_district']=='Homewood City','True',
                                                        np.where(df_flat_file['updated_district']=='Mobile County','True',
                                                                 np.where(df_flat_file['updated_district']=='Montgomery County','True',
                                                                          np.where(df_flat_file['updated_district']=='Perry County','True',
                                                                                   np.where(df_flat_file['updated_district']=='Sumter County','True','False'))))))


df_flat_file

In [ ]:
df_flat_file[df_flat_file['system_name']==df_flat_file['school_name']]

In [ ]:
df_flat_file.columns

In [ ]:
df_flat_file = (
    df_flat_file
    .merge(df_assessment_2021_all_students_pivot, on=['system_name','school_name'], how='left')
)

df_flat_file = df_flat_file.drop_duplicates()
df_flat_file

In [ ]:
df_flat_file['2021_2022_ela_change'] = df_flat_file['2022_percent_proficient_ela']-df_flat_file['2021_percent_proficient_ela']
df_flat_file['2021_2022_math_change'] = df_flat_file['2022_percent_proficient_math']-df_flat_file['2021_percent_proficient_math']

In [ ]:
df_flat_file.to_csv('../data_for_analysis/2022_assessment_flat_file.csv', encoding='utf-8', index=False)

In [ ]:
df_flat_file.loc[df_flat_file['system_code']==800]

In [ ]:
df_assessment_2022_all_students_pivot.loc[df_assessment_2022_all_students_pivot['system_name']=='i3 Academy']

## Demographics

In [ ]:
df_demographics_2021

In [ ]:
df_list = ['df_demographics_2022',
                  'df_demographics_2021',
                  'df_demographics_2020',
                  'df_demographics_2019',
                  'df_demographics_2018',
                  'df_demographics_2017',
                  'df_demographics_2016',
                  'df_demographics_2015']
for d in df_list:
    # globals()[d]['is_charter'] = 'False'
    # globals()[d]['is_charter'] = (
    #     globals()[d]['site_type']
    #     .replace('Public Charter School - Converted','True')
    #     .replace('Public Charter School Startup','True')
    #     .replace('Public Regular School','False')
    #     .replace('Public Virtual School','False')
    #     .replace('Public Magnet School','False')
    #     .replace('Public Alternative School','False')
    #     .replace('Public Special Education School','False')
    #     .replace('Public Board of Education',np.NaN)
    #     .replace('Central Office',np.NaN)
    #     .replace('Regular School with Magnet Program','False')
    #     .replace('Special Education School (No Detail)','False')
    #     .replace('Special Education - Private Services','False')
    #     .replace('Public Career & Technical Center','False')
    #     .replace('State Supported School Other Facility','False')
    # )
    
    globals()[d] = globals()[d].loc[(globals()[d]['grade']=='All Grades')
                       & (globals()[d]['gender']=='All Gender')
                       & (globals()[d]['ethnicity']=='All Ethnicity')
                       & (globals()[d]['sub_population']=='All SubPopulation')]
    
    globals()[d]['file_year'] = d.split('_')[-1]
    
df_demographics = pd.concat([df_demographics_2022,
                          df_demographics_2021,
                          df_demographics_2020,
                          df_demographics_2019,
                          df_demographics_2018,
                          df_demographics_2017,
                          df_demographics_2016,
                          df_demographics_2015]).drop_duplicates()

df_demographics.to_csv('../raw_data_files_clean/merged_demographics.csv', encoding='utf-8', index=False)

In [ ]:
df_demographics[df_demographics['system_name']=='i3 Academy']

In [ ]:
df_demographics_2020.loc[df_demographics_2020['system_name']=='LEAD Academy']

### Graduation

In [ ]:
for a, n in zip([df_grad_2022,df_grad_2021,df_grad_2020,df_grad_2019,df_grad_2018], 
                ['df_grad_2022','df_grad_2021','df_grad_2020','df_grad_2019','df_grad_2018']):
    s = a.copy()
    s['school_name'] = (
        s['school_name']
        .replace('i3 Academy','i3 Academy - All')
        .replace('LEAD Academy','LEAD Academy - All')
        
    )
    s['file_source'] = n
    s['file_year'] = n.split('_')[-1]
    vars()[n] = s

In [ ]:
df_grad_2022

In [ ]:
df_grad = pd.concat([df_grad_2022,df_grad_2021,df_grad_2020,df_grad_2019,df_grad_2018])

df_merged_grad = pd.merge(df_grad, df_directory, on=['system_name', 'school_name', 'file_year'], how='left')

In [ ]:
df_grad_all = df_merged_grad.loc[((df_merged_grad['is_charter'].notnull()) | (df_merged_grad['system_name']=='Alabama State Department of Education'))
                  & (df_merged_grad['grade']=='All Grades')
                  & (df_merged_grad['gender']=='All Gender')
                  & (df_merged_grad['race']=='All Race')
                  & (df_merged_grad['ethnicity']=='All Ethnicity')
                  & (df_merged_grad['sub_population']=='All SubPopulation')].drop_duplicates()
df_grad_all['sub_group'] = 'All'

df_grad_black = df_merged_grad.loc[((df_merged_grad['is_charter'].notnull()) | (df_merged_grad['system_name']=='Alabama State Department of Education'))
                  & (df_merged_grad['grade']=='All Grades')
                  & (df_merged_grad['gender']=='All Gender')
                  & (df_merged_grad['race']=='Black or African American')
                  & (df_merged_grad['ethnicity']=='Other Ethnicity')
                  & (df_merged_grad['sub_population']=='All SubPopulation')].drop_duplicates()
df_grad_black['sub_group'] = 'Black'

df_grad_hispanic = df_merged_grad.loc[((df_merged_grad['is_charter'].notnull()) | (df_merged_grad['system_name']=='Alabama State Department of Education'))
                  & (df_merged_grad['grade']=='All Grades')
                  & (df_merged_grad['gender']=='All Gender')
                  & (df_merged_grad['race']=='All Race')
                  & (df_merged_grad['ethnicity']=='Hispanic/Latino')
                  & (df_merged_grad['sub_population']=='All SubPopulation')
                  ].drop_duplicates()
df_grad_hispanic['sub_group'] = 'Hispanic'

df_grad_econ_disadv = df_merged_grad.loc[((df_merged_grad['is_charter'].notnull()) | (df_merged_grad['system_name']=='Alabama State Department of Education'))
                  & (df_merged_grad['grade']=='All Grades')
                  & (df_merged_grad['gender']=='All Gender')
                  & (df_merged_grad['race']=='All Race')
                  & (df_merged_grad['ethnicity']=='All Ethnicity')
                  & (df_merged_grad['sub_population']=='Economically Disadvantaged')
                  ].drop_duplicates()
df_grad_econ_disadv['sub_group'] = 'Economically Disadvantaged'


In [ ]:
grad_subgroup = pd.concat([df_grad_all,df_grad_black,df_grad_hispanic,df_grad_econ_disadv])
grad_subgroup

In [ ]:

grad_subgroup.to_csv('../raw_data_files_clean/merged_grad.csv', encoding='utf-8', index=False)